<a href="https://colab.research.google.com/github/rrbauman/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-14 21:30:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-14 21:30:32 (9.89 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date



In [18]:
df.count()

2166601

In [6]:
dropna_df = df.dropna()
dropna_df = dropna_df.distinct()
dropna_df.count()

1785886

In [7]:
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010D6B1QM7U|          1|            0|          0|   N|                Y|
|R100BI61LIKH2N|          5|            1|          2|   N|                Y|
|R100TPJCYDYGBU|          3|            1|          2|   N|                Y|
|R1018OO9U6UUWO|          5|            0|          0|   N|                Y|
|R102YIWZEIAXT9|          5|            2|          4|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
vine_total_votes_df= vine_df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_total_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import col
vine_percents_df=vine_total_votes_df.withColumn('percent', col("helpful_votes")/ col("total_votes"))
vine_percents_df=vine_percents_df.filter('percent>=0.5')
vine_percents_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|0.9148936170212766|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|0.9347826086956522|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|0.8363636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|0.5714285714285714|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|             0.905|
| RZAE1OYOXEOAR|          5|           86|         94|   N|                Y|0.9148936170212766|
|R1TBFM7966BSCZ|          1|           33|         56|   N|                Y|0.5892857142857143|
|R3NSDYMPN0WV6M|          4|  

In [11]:
vine_Y_df=vine_percents_df.filter(col("vine").contains("Y"))
vine_Y_df.count()

94

In [12]:
vine_N_df=vine_percents_df.filter(col("vine").contains("N"))
vine_N_df.count()


40471

In [33]:
vine_Y_df.count()

94

In [13]:
vine_Y_df.filter(vine_Y_df.star_rating == 5).count()

48

In [15]:
vine_N_df.filter(vine_N_df.star_rating == 5).count()


15663